# **World Bank API**

<div align="justify">
The World Bank API provides access to files, databases, and metadata related to global indicators such as population statistics, health systems, economic performance, education, social inclusion, and environmental factors across different countries.

For this pipeline, data was retrieved from several indicators of previous fields for three North American countries: Canada, the United States, and Mexico. The data was then merged and cleaned to ensure a consistent structure across all indicators. As part of the process, the dataset was briefly analyzed to identify missing values and overall data quality.

Finally, the cleaned dataset was stored in MySQL using MySQL Workbench. This exercise represents a complete data engineering workflow, covering data extraction, transformation, validation, storage, and movement between systems.
</div>


In [ ]:
import matplotlib.pyplot as plt # Visualizations
import pandas as pd # Dataframes
import requests # API connection

In [2]:
# Define relevant variables to consider
countries = {
    'MEX': 'Mexico',
    'CAN': 'Canada',
    'USA': 'United States of America'
  }


# Select indicators
indicators = {
    
  # Population (SP)
  "SP.DYN.LE00.IN": "life_expectancy",
  "SP.DYN.IMRT.IN": "infant_mortality",
  "SP.DYN.CDRT.IN": "death_rate",
  "SP.DYN.TFRT.IN": "fertility_rate",
  "SP.POP.TOTL": "population_total",

  # Health systems (SH)
  "SH.XPD.CHEX.GD.ZS": "health_expenditure_pct_gdp",
  "SH.MED.BEDS.ZS": "hospital_beds_per_1000",

  # Economy (NY)
  "NY.GDP.MKTP.CD": "gdp_usd",
  "NY.GDP.PCAP.CD": "gdp_per_capita_usd",
  "NY.GDP.MKTP.KD.ZG": "gdp_growth_percent",
  "NY.GNP.PCAP.CD": "gnp_per_capita_usd",
  "NY.INC.PCAP.CD": "gni_per_capita_usd",
  "NY.EXP.GNFS.ZS": "exports_percent_gdp",
  "NY.IMP.GNFS.ZS": "imports_percent_gdp",
  "FP.CPI.TOTL.ZG": "inflation_percent",
  "SL.UEM.TOTL.ZS": "unemployment_percent",

  # Education (SE)
  "SE.ADT.LITR.ZS": "literacy_rate",
  "SE.XPD.TOTL.GD.ZS": "education_expenditure_pct_gdp",

  # Social inclusion / Poverty (SI)
  "SP.POP.TOTL": "population_total",
  "SP.POP.GROW": "population_growth_percent",
  "SI.POV.DDAY": "poverty_headcount_1_90",
  "SI.DST.04TH.20": "income_share_top_20",

  # Environment (EN)
  "EN.CLC.MDAT.ZS": "climate_disaster_impact_percent",
}


In [3]:
# Function to fetch a specific country with a specific indicator
def fetch_indicator(country_code, ind_code, ind_name):

    params = {"format": "json", 
              "page": 1}

    # URL 
    url = f"https://api.worldbank.org/v2/countries/{country_code}/indicators/{ind_code}"

    response = requests.get(url, params=params)

    # Obtain data
    data = response.json()

    # Check if data was returned by the API (not empty)

    # Data is an array with dictionaries, the first element is the metadata and the 
    # second is the data itself
    if len(data) < 2 or data[1] is None:
        return pd.DataFrame()

    df = pd.DataFrame(data[1]) # Convert into a dataframe
    df = df[["countryiso3code", "date", "value"]] # Select columns

    # Change column names
    df.columns = ["country_code", "year", ind_name]

    return df


In [4]:
# Fetch all indicators for a single country
def retrieve_data_country(country_code):
    base_df = pd.DataFrame()

    # Iterate over the indicator codes and names
    for ind_code, ind_name in indicators.items():

        # Checking if data is loading (debbuging)
        # print(f"Fetching {ind_name}")

        ind_df = fetch_indicator(country_code, ind_code, ind_name)

        # Skip if no data was returned
        if ind_df.empty:
            continue

        # Initialize dataframe with the first indicator
        if base_df.empty:
            base_df = ind_df
            continue

        # Add column to the left only if it does not exist
        if ind_name not in base_df.columns:
            base_df = base_df.merge(
                ind_df,
                on=["country_code", "year"],
                how="left"
            )

    return base_df


In [5]:
# Build a unified dataset for all selected countries
def merged_database(countries):

    # Empty list to storage 
    dfs_array = []

    for country_code in countries.keys():
        dfs_array.append(retrieve_data_country(country_code))

    # Combine all countries into a single df
    final_database = pd.concat(dfs_array, ignore_index = True)

    return final_database
        

health_df = merged_database(countries)
health_df


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14016\1490966850.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_database = pd.concat(dfs_array, ignore_index = True)


,country_code,year,life_expectancy,infant_mortality,death_rate,fertility_rate,population_total,health_expenditure_pct_gdp,hospital_beds_per_1000,gdp_usd,...,gdp_growth_percent,gnp_per_capita_usd,inflation_percent,unemployment_percent,literacy_rate,education_expenditure_pct_gdp,population_growth_percent,poverty_headcount_1_90,income_share_top_20,climate_disaster_impact_percent
0,MEX,2024,NaN,NaN,NaN,NaN,130861007,NaN,NaN,1.856366e+12,...,1.427428,12850,4.722256,2.711,95.765472,NaN,0.860515,NaN,NaN,NaN
1,MEX,2023,75.069000,10.8,6.161,1.910,129739759,NaN,NaN,1.798318e+12,...,3.353541,12020,5.527961,2.765,95.834690,NaN,0.872179,NaN,NaN,NaN
2,MEX,2022,73.973000,11.1,6.521,1.940,128613117,5.721452,NaN,1.466935e+12,...,3.709757,10730,7.896276,3.256,95.848660,4.06586,0.753117,2.3,20.7,NaN
3,MEX,2021,69.750000,11.5,9.015,1.970,127648148,5.892443,1.02,1.316569e+12,...,6.048483,9880,5.689208,4.019,95.689098,4.23823,0.667405,NaN,NaN,NaN
4,MEX,2020,70.449000,11.8,8.429,1.990,126799054,6.053740,1.00,1.121065e+12,...,-8.354035,8920,3.396834,4.440,95.250000,4.49967,0.820454,4.3,20.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,USA,1979,73.804878,13.1,8.500,1.808,225055000,NaN,6.10,2.627333e+12,...,3.165988,12200,11.254471,NaN,NaN,NaN,1.103577,0.7,23.5,NaN
146,USA,1978,73.356098,13.8,8.700,1.760,222585000,NaN,6.20,2.351599e+12,...,5.535206,10760,7.630964,NaN,NaN,NaN,1.059573,0.7,23.6,NaN
147,USA,1977,73.256098,14.4,8.600,1.790,220239000,NaN,6.40,2.081826e+12,...,4.624187,9590,6.501684,NaN,NaN,NaN,1.005772,0.5,23.4,NaN
148,USA,1976,72.856098,15.2,8.800,1.738,218035000,NaN,6.60,1.873412e+12,...,5.388034,8960,5.744813,NaN,NaN,NaN,0.950220,0.7,23.5,NaN


In [ ]:
# Cleaning and validatiing step for the pipeline after exploring the dataset
def clean_validate_dataset(df):

    df = df.copy()

    # Standardize column names
    df.columns = df.columns.str.lower()

    # Ensure year is numeric
    df["year"] = pd.to_numeric(df["year"], errors="coerce")

    # Replace empty strings with NaN
    df = df.replace("", pd.NA)

    # Convert numeric columns (except identifiers)
    for col in df.columns:
        if col not in ["country_code"]:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Drop rows without key identifiers
    before_rows = len(df)
    df = df.dropna(subset=["country_code", "year"])
    after_rows = len(df)

    print(f"Dropped rows with missing keys: {before_rows - after_rows} ")

    df = df.sort_values(["country_code", "year"]) # Sort for consistency (and better-looking)

    df = df.reset_index(drop=True) # Reset index after cleaning and sorting, so it doesn't look weird
    
    df = df.rename(columns = {'year': 'date_year'})

    
    ### ---- Print data quality checks ----

    # Check duplicate country-year records
    duplicates = df.duplicated(subset=["country_code", "date_year"]).sum()
    print(f"Duplicate country-year rows: {duplicates}\n")

    # Check missing values per column
    missing_summary = df.isna().sum()
    print("\nMissing values per column:\n")
    print(missing_summary[missing_summary > 0])


    return df


In [27]:
final_df = clean_validate_dataset(health_df)

Dropped rows with missing keys: 0 
Duplicate country-year rows: 0


Missing values per column:

life_expectancy                      3
infant_mortality                     3
death_rate                           3
fertility_rate                       3
health_expenditure_pct_gdp          80
hospital_beds_per_1000              27
unemployment_percent                48
literacy_rate                      125
education_expenditure_pct_gdp       71
poverty_headcount_1_90              39
income_share_top_20                 39
climate_disaster_impact_percent    147
dtype: int64


In [28]:
final_df

,country_code,date_year,life_expectancy,infant_mortality,death_rate,fertility_rate,population_total,health_expenditure_pct_gdp,hospital_beds_per_1000,gdp_usd,...,gdp_growth_percent,gnp_per_capita_usd,inflation_percent,unemployment_percent,literacy_rate,education_expenditure_pct_gdp,population_growth_percent,poverty_headcount_1_90,income_share_top_20,climate_disaster_impact_percent
0,CAN,1975,73.521707,14.1,7.30,1.8240,23143275,NaN,NaN,1.744198e+11,...,1.545403,8020,10.672189,NaN,NaN,6.42132,1.459425,1.7,23.4,NaN
1,CAN,1976,73.856098,13.3,7.20,1.7960,23449808,NaN,6.90,2.072716e+11,...,5.889276,8780,7.541739,NaN,NaN,6.69842,1.315807,NaN,NaN,NaN
2,CAN,1977,74.215610,12.5,7.10,1.7820,23725843,NaN,6.90,2.123252e+11,...,3.534827,9130,7.976445,NaN,NaN,6.71092,1.170257,1.5,23.6,NaN
3,CAN,1978,74.529756,11.7,7.10,1.7680,23963203,NaN,6.90,2.193695e+11,...,3.735976,9750,8.973723,NaN,NaN,7.07383,0.995457,NaN,NaN,NaN
4,CAN,1979,74.866341,11.0,7.00,1.7540,24201544,NaN,6.80,2.438911e+11,...,3.725199,10600,9.144677,NaN,NaN,6.69220,0.989699,1.2,23.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,USA,2020,76.980488,5.5,10.27,1.6415,331577720,18.813253,2.74,2.106047e+13,...,-2.163029,64920,1.233584,8.055,NaN,5.39532,0.408428,0.5,22.7,NaN
146,USA,2021,76.329268,5.5,10.40,1.6640,332099760,17.506386,NaN,2.331508e+13,...,6.055053,71730,4.697859,5.349,NaN,5.42038,0.157317,0.5,22.2,NaN
147,USA,2022,77.434146,5.5,9.80,1.6565,334017321,16.496140,NaN,2.560485e+13,...,2.512375,77330,8.002800,3.650,NaN,NaN,0.575745,1.2,22.6,NaN
148,USA,2023,78.385366,5.5,9.20,1.6165,336806231,NaN,NaN,2.729217e+13,...,2.887556,80000,4.116338,3.638,NaN,NaN,0.831493,1.2,22.7,NaN


In [29]:
# Save final clean dataset to CSV
output_path = "worldbank_health_economy_clean.csv"

final_df.to_csv(output_path,index=False)

# Print info
print(f"Clean dataset saved to {output_path}!")

Clean dataset saved to worldbank_health_economy_clean.csv!
